In [20]:
# Request entire 2025 season data

from enum import IntEnum
import os
import requests
import time
import json
import math
import random
import pandas as pd


# --- Configuration ---
JSON_FOLDER = 'pitch_raw'
CSV_FOLDER = 'pitch_processed'
JSON_SUFFIX = 'raw'
CSV_SUFFIX = 'processed'

RECORD_URL = "https://api-gw.sports.naver.com/schedule/games/{}/record"
RELAY_URL = "https://api-gw.sports.naver.com/schedule/games/{}/relay?inning={}"

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
}

class TextType(IntEnum):
    INNING_START = 0 # including start of half inning
    PITCH = 1
    SUBSTITUTION = 2
    TIMEOUT = 7 # Mound visit, VAR, pitcher leave mound
    PA_START = 8
    PA_RESULT_SELF = 13 # BB, HbP, Double play self out
    PA_RESULT_RUNNER = 14 # BB, Steal, Double play runner out, runner base run
    PA_RESULT_RBI_SELF = 23 # Hit, BB, 
    PA_RESULT_RBI_RUNNER = 24 # Hit, BB
    INNING_END = 99
# ---------------------
    
def _make_safe_request(url, max_retries=5):
    """
    Performs a request with anti-blocking measures: random delay and exponential backoff.
    Returns the Response object on success, or None on failure.
    """
    # Start with a conservative wait time
    wait_time = 2  
    
    for attempt in range(max_retries):
        try:
            # 1. Randomized Delay (The critical anti-blocking measure)
            delay = random.uniform(2.5, 9.5)
            # print(f"Delaying for {delay:.2f} seconds...") # Optional: Debug print
            time.sleep(delay)

            # 2. Make the request with standard headers
            response = requests.get(url, headers=HEADERS, timeout=30)
            
            # 3. Handle success
            if response.status_code == 200:
                return response
            
            # 4. Handle "Too Many Requests" (429) using exponential backoff
            elif response.status_code == 429:
                print(f"[{url}] Received 429 on attempt {attempt + 1}. Waiting {wait_time}s and retrying.")
                time.sleep(wait_time)
                wait_time *= 2  # Double the wait time (5, 10, 20, ...)
            
            # 5. Handle other HTTP errors (404, 500, etc.)
            else:
                response.raise_for_status() # Raises an exception for 4xx/5xx status codes
                
        except requests.exceptions.RequestException as e:
            print(f"[{url}] Request failed on attempt {attempt + 1}: {e}")
            if attempt < max_retries - 1:
                # Wait before retrying on general network errors
                time.sleep(wait_time)
            
        except Exception as e:
             # Handle unexpected exceptions
             print(f"[{url}] Unexpected error: {e}")
             break

    print(f"[{url}] Failed to retrieve data after {max_retries} attempts.")
    return None

def _extract_game_data(data):
    """Safely extracts max_inning and PA list from the JSON data."""
    text_relay_data = data.get('result', {}).get('textRelayData', {})
    
    # max_inning needs to be extracted as an integer
    try:
        max_inning = int(text_relay_data.get('inn', 0))
    except (TypeError, ValueError):
        max_inning = 0
        
    pa_list = text_relay_data.get('textRelays', [])

    # REVERSE THE LIST FOR CHRONOLOGICAL ORDER
    pa_list.reverse() 
    
    # The full data object is needed later to build the pitcher lookup (from inn 1 data)
    return max_inning, pa_list
    
def get_record_json_file(game_id):
    os.makedirs(JSON_FOLDER, exist_ok=True)
    raw_file_path = os.path.join(JSON_FOLDER, f'{game_id}_record_{JSON_SUFFIX}.json')
    data = None
    
    # 1. CHECK FOR EXISTING FILE
    if os.path.exists(raw_file_path):
        try:
            with open(raw_file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
        except json.JSONDecodeError as e:
            print(f"Error loading existing JSON file {raw_file_path}: {e}. Proceeding to refetch.")
            data = None # Set to None to trigger refetch below

    # 2. FETCH NEW DATA if necessary
    if data is None:
        url = RECORD_URL.format(game_id)

        response = _make_safe_request(url)

        if response and response.status_code == 200:
            try:
                data = response.json()
                
                # Save the Raw JSON for the inning
                with open(raw_file_path, 'w', encoding='utf-8') as f:
                    json.dump(data, f, ensure_ascii=False, indent=4)
                print(f"  -> Successfully fetched and saved raw JSON to {raw_file_path}")
                    
            except json.JSONDecodeError:
                print(f"Error decoding JSON for record.")
        else:
            # The request failed or returned a non-200/429 status after max retries
            print(f"Failed to retrieve data for record.")

    # 3. PROCESS DATA (both loaded and fetched)
    if data:
        record_data = data.get('result', {}).get('recordData', {})
        batter_record_lookup, pitcher_record_lookup = _build_record_lookup(record_data)
    
    return batter_record_lookup, pitcher_record_lookup

def get_json_files(game_id):
    """
    Fetches game data by inning, checks for existing files, and merges PA data.
    Returns: master_json_list (list of all PAs) and pitcher_lookup.
    """
    os.makedirs(JSON_FOLDER, exist_ok=True)
    inning_counter = 1
    max_inning = 1
    master_json_list = []
    pitcher_lookup = None

    while inning_counter <= max_inning:
        raw_file_path = os.path.join(JSON_FOLDER, f'{game_id}_{inning_counter}_{JSON_SUFFIX}.json')
        data = None
        fetched_new_data = False

        # 1. CHECK FOR EXISTING FILE
        if os.path.exists(raw_file_path):
            try:
                with open(raw_file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
            except json.JSONDecodeError as e:
                print(f"Error loading existing JSON file {raw_file_path}: {e}. Proceeding to refetch.")
                data = None # Set to None to trigger refetch below

        # 2. FETCH NEW DATA if necessary
        if data is None:
            print(f"Fetching Inning {inning_counter}...")
            url = RELAY_URL.format(game_id, inning_counter)

            response = _make_safe_request(url)

            if response and response.status_code == 200:
                try:
                    data = response.json()
                    fetched_new_data = True
                    
                    # Save the Raw JSON for the inning
                    with open(raw_file_path, 'w', encoding='utf-8') as f:
                        json.dump(data, f, ensure_ascii=False, indent=4)
                    print(f"  -> Successfully fetched and saved raw JSON to {raw_file_path}")
                        
                except json.JSONDecodeError:
                    print(f"Error decoding JSON for Inning {inning_counter}. Skipping.")
                    break # Stop on corrupted response
            else:
                # The request failed or returned a non-200/429 status after max retries
                print(f"Failed to retrieve data for Inning {inning_counter} after all retries. Stopping.")
                break

        # 3. PROCESS DATA (both loaded and fetched)
        if data:
            current_max_inning, pa_list = _extract_game_data(data)
            
            if inning_counter == 1:
                # Update max_inning based on the first inning's data
                max_inning = current_max_inning
                pitcher_lookup = _build_pitcher_lookup(data)
                print(f"Max inning set to: {max_inning}")

            if max_inning == 0:
                print(f"Unable to get a valid max_inning (is 0). Stopping.")
                break
                
            if not pa_list:
                print(f"Inning {inning_counter} contains no Plate Appearances. Stopping.")
                break
            
            # Collect the 'textRelays' list for later merging
            master_json_list.extend(pa_list)
            inning_counter += 1

    print(f"\nFinished data collection. Total PA lists collected: {len(master_json_list)}")
    return master_json_list, pitcher_lookup
    
def extract_team_codes(game_id):
    """
    Extracts the 2-character away and home team codes from the game_id string.
    Example: '20250930LTHH02025' -> Away: 'LT', Home: 'HH'
    """
    if len(game_id) < 12:
        return 'N/A', 'N/A' # Handle unexpected format
        
    away_code = game_id[8:10]
    home_code = game_id[10:12]
    
    return away_code, home_code
    
def _build_record_lookup(data):
    """
    Returns:
        dict: dictionary mapping player 'pcode' to their record data
    """
    batter_record_lookup = {}
    pitcher_record_lookup = {}
    
    batter_record = data.get('battersBoxscore', {})
    pitcher_record = data.get('pitchersBoxscore', {})
    away_batter_record = batter_record.get('away', [])
    home_batter_record = batter_record.get('home', [])
    away_pitcher_record = pitcher_record.get('away', [])
    home_pitcher_record = pitcher_record.get('home', [])
    
    for p in away_batter_record:
        pcode = p.get('playerCode')
        if pcode:
            batter_record_lookup[pcode] = p
        
    for p in home_batter_record:
        pcode = p.get('playerCode')
        if pcode:
            batter_record_lookup[pcode] = p
        
    for p in away_pitcher_record:
        pcode = p.get('playerCode')
        if pcode:
            pitcher_record_lookup[pcode] = p
        
    for p in home_pitcher_record:
        pcode = p.get('playerCode')
        if pcode:
            pitcher_record_lookup[pcode] = p
        
        return batter_record_lookup, pitcher_record_lookup

def _build_pitcher_lookup(full_inning_1_data):
    """
    Extracts pitcher data from the full Inning 1 game object and creates a lookup dictionary.

    Args:
        full_inning_1_data (dict): The full JSON object from the first inning fetch.

    Returns:
        dict: A dictionary mapping pitcher 'pcode' to their static data (name, stance, etc.).
    """
    pitcher_lookup = {}
    
    try:
        data = full_inning_1_data['result']['textRelayData']
        home_pitcher_list = data['homeLineup']['pitcher']
        away_pitcher_list = data['awayLineup']['pitcher']
        all_pitchers_list = away_pitcher_list + home_pitcher_list
        
        for p in all_pitchers_list:
            pcode = p.get('pcode')
            if pcode:
                # Pre-calculate stance for efficiency
                hit_type = p.get('hitType', 'L')
                p['stance_derived'] = 'R' if hit_type.startswith('우') else 'L'
                pitcher_lookup[pcode] = p
        
        return pitcher_lookup
        
    except (KeyError, TypeError) as e:
        print(f"Error building pitcher lookup from lineup data: {e}")
        return {}

# --- KINEMATICS AND ZONE CLASSIFICATION FUNCTIONS ---

def calculate_plate_height(pitch_data):
    """Calculates the flight time and the corrected vertical position (z_plate)."""
    try:
        y0 = pitch_data['y0']
        vy0 = pitch_data['vy0']
        ay = pitch_data['ay']
        z0 = pitch_data['z0']
        vz0 = pitch_data['vz0']
        az = pitch_data['az']
    except KeyError as e:
        # print(f"Missing required key in pitch data: {e}") # Suppress during bulk processing
        return None

    # Step 1: Calculate Flight Time (t)
    a = 0.5 * ay
    b = vy0
    c = y0
    discriminant = (b**2) - (4 * a * c)
    
    if discriminant < 0 or a == 0:
        return {"error": "Invalid kinematics data."}
    
    # Use the minus branch for the time to the plate (t > 0)
    time_of_flight = (-b - math.sqrt(discriminant)) / (2 * a)
    
    # Step 2: Calculate Vertical Position (z_plate)
    z_plate = z0 + (vz0 * time_of_flight) + (0.5 * az * (time_of_flight**2))

    return {
        "time_of_flight": time_of_flight,
        "z_plate": z_plate
    }

def classify_5x5_zone(crossPlateX, calculated_z, topSz, bottomSz):
    """
    Classifies a pitch into one of 25 zones (11 to 55) based on a 5x5 grid 
    (3x3 zone + 1 block shadow), and returns a flag if it is outside even the shadow.
    """
    
    # 1. Define Standard Baseball Constants
    PLATE_WIDTH_FT = 1.4167  # 17 inches
    HALF_PLATE = PLATE_WIDTH_FT / 2 # 0.70835
    
    # 2. Define Zone Block Sizes
    X_BLOCK = PLATE_WIDTH_FT / 3
    Z_BLOCK = (topSz - bottomSz) / 3
    
    # 3. Define X Boundaries (6 boundaries create 5 zones)
    x_boundaries = [
        -HALF_PLATE - X_BLOCK,   # X1: Far Left Shadow boundary
        -HALF_PLATE,             # X2: Left edge of plate
        -HALF_PLATE + X_BLOCK,   # X3: Left-center boundary
        HALF_PLATE - X_BLOCK,    # X4: Right-center boundary
        HALF_PLATE,              # X5: Right edge of plate
        HALF_PLATE + X_BLOCK     # X6: Far Right Shadow boundary
    ]
    
    # 4. Define Z Boundaries (6 boundaries create 5 zones)
    z_boundaries = [
        bottomSz - Z_BLOCK,      # Z1: Far Low Shadow boundary
        bottomSz,                # Z2: Bottom of Zone
        bottomSz + Z_BLOCK,      # Z3: Low-mid boundary
        topSz - Z_BLOCK,         # Z4: Mid-high boundary
        topSz,                   # Z5: Top of Zone
        topSz + Z_BLOCK          # Z6: Far High Shadow boundary
    ]
    
    # Determine the X-Index -1, (0 to 4), 5
    x_index = -1
    is_out_bound = False
    for i, boundary in enumerate(x_boundaries):
        if crossPlateX < boundary:
            x_index = i
            break
    else:
        x_index = 5

    # Determine the Z-Index -1, (0 to 4), 5
    z_index = -1
    for i, boundary in enumerate(z_boundaries):
        if calculated_z < boundary:
            z_index = i
            break
    else:
        z_index = 5


    # --- Determine Outside Boundary Flag ---
    # The flag is True if the pitch is outside the 5x5 grid (index 0 or 5)
    is_outside_boundary = (x_index == 0) or (x_index == 5) or \
                              (z_index == 0) or (z_index == 5)
    
    # --- Calculate Final Zone ID (clamping index to 1-5) ---
    # Clamp the index between 1 and 5 (index 0 maps to zone 1, index 5 maps to zone 5)
    # This prevents the final Zone ID from being 00 or 66, but still flags the issue.
    # Note: Zone ID 1x is the low row, 5x is the high row.
    final_z_index = max(1, min(z_index, 5))
    final_x_index = max(1, min(x_index, 5))

    # Zone ID (e.g., Row 1 x 10 + Col 1 = 11, Row 5 x 10 + Col 5 = 55)
    zone_id = final_z_index * 10 + final_x_index
    
    return {
        "zone_5x5_id": str(zone_id),
        "is_outside_boundary": is_outside_boundary,
        "raw_x_index": x_index,
        "raw_z_index": z_index
    }
    
def process_plate_appearance(pa_data, pitcher_lookup, away_code, home_code, batter_record_lookup, pitcher_record_lookup):
    """
    Processes all events
    within a single Plate Appearance (PA), correctly linking context to pitch data.
    Returns a list of dictionaries, one for each processed pitch (type 1).
    """
    processed_pitches_in_pa = []

    # Create a MAP for Trajectory Data (ptsOptions)
    # Key: pitchId (e.g., "251007_151509")
    # Value: The full trajectory dict
    trajectory_map = {
        pitch.get('pitchId'): pitch
        for pitch in pa_data.get('ptsOptions', [])
    }
    
    # Contextual variables to be updated by Type 8 events
    current_batter_name = 'N/A'
    current_batter_lineup_pos = 'N/A'
    current_batter_id = 'N/A' 
    is_batter_home = pa_data.get('homeOrAway') == "1"

    # Determine static PA team codes
    batter_team_code = home_code if is_batter_home else away_code
    pitcher_team_code = away_code if is_batter_home else home_code

    # Check for pitcher change SUBSTITUTION
    #pitcher_data = pitcher_lookup.get(pitcher_id, {})
    #pitcher_name = pitcher_data.get('name', 'N/A')
    #pitcher_stance = pitcher_data.get('stance_derived', 'L') # Using pre-derived stance

    text_options = pa_data.get('textOptions', [])
    pa_result_long = 'N/A'
    pa_result_base1 = 'N/A'
    pa_result_base2 = 'N/A'
    pa_result_base3 = 'N/A'
    pa_result_runs = 'N/A'
    
    # Iterate backward through Text Events (to get results)
    for detail in reversed(text_options):
        event_type = detail.get('type')
        match event_type:
            # even if inning ends not because of batter, need to record pa_result_base?
            case TextType.PA_RESULT_RUNNER:
                #pa_result = detail.get('text')
                pa_result = ''
                
                currentGameState = detail.get('currentGameState', {})
                pa_result_base1 = currentGameState.get('base1')
                pa_result_base2 = currentGameState.get('base2')
                pa_result_base3 = currentGameState.get('base3')
                continue
            case TextType.PA_RESULT_SELF | TextType.PA_RESULT_RBI_SELF:
                pa_result_long = detail.get('text').split(": ",1)[1]
                
                currentGameState = detail.get('currentGameState', {})
                pa_result_base1 = currentGameState.get('base1')
                pa_result_base2 = currentGameState.get('base2')
                pa_result_base3 = currentGameState.get('base3')
                break
    
    # Iterate forward through Text Events (textOptions)
    for detail in text_options:
        event_type = detail.get('type')

        # EVENT TYPE 8: BATTER CHANGE (Context Update)
        match event_type:
            case TextType.PA_START:
                batter_record = detail.get('batterRecord', {})
                current_batter_name = batter_record.get('name', 'N/A')
                current_batter_lineup_pos = batter_record.get('batOrder', 'N/A')
                current_batter_id = batter_record.get('pcode', 'N/A')
                continue

            # EVENT TYPE 1: PITCH EVENT (Data Processing)
            case TextType.PITCH:
                pitch_id = detail.get('ptsPitchId')
                pitch = trajectory_map.get(pitch_id)
    
                if pitch is None:
                    # This pitch event exists in textOptions but has NO kinematic data.
                    # We skip processing it as a tracked pitch, which is correct.
                    # print(f"Skipping pitch {pitch_id}: No trajectory data found.")
                    continue
                    
                pitch_summary = {}
    
                # --- A. MERGE DATA & CALCULATIONS ---
                
                # Add Trajectory Data (ptsOptions)
                pitch_summary.update(pitch)
    
                # Kinematic Calculations
                calculation_result = calculate_plate_height(pitch)
                
                pitch_summary['plate_z_ft'] = None
                zone_5x5_id = 0
                is_outside_boundary = False
    
                if calculation_result:
                    calculated_z = calculation_result.get('z_plate')
                    zone_results = classify_5x5_zone(
                        pitch.get('crossPlateX', 0.0), calculated_z, 
                        pitch.get('topSz', 3.3), pitch.get('bottomSz', 1.6)
                    )
                    pitch_summary['plate_z_ft'] = round(calculated_z, 4)
                    zone_5x5_id = zone_results['zone_5x5_id']
                    is_outside_boundary = zone_results['is_outside_boundary']
    
                # --- B. ADD CONTEXTUAL & PITCH DATA ---
                
                currentGameState = detail.get('currentGameState', {})
                pitcher_id = currentGameState.get('pitcher')
                pitcher_data = pitcher_lookup.get(pitcher_id, {})
    
                # 1. Game State
                pitch_summary['is_batter_home'] = is_batter_home
                pitch_summary['home_score'] = currentGameState.get('homeScore')
                pitch_summary['away_score'] = currentGameState.get('awayScore')
                pitch_summary['strike'] = currentGameState.get('strike')
                pitch_summary['ball'] = currentGameState.get('ball')
                pitch_summary['out'] = currentGameState.get('out')
                pitch_summary['base1'] = currentGameState.get('base1')
                pitch_summary['base2'] = currentGameState.get('base2')
                pitch_summary['base3'] = currentGameState.get('base3')
    
                # 3. Pitcher Info (From currentGameState)
                pitch_summary['pitcher_id'] = pitcher_id
                hit_type = pitcher_data.get('hitType', 'L') # Check for the '우' (Right) character in 'hitType' X투X타
                pitch_summary['pitcher_stance'] = 'R' if hit_type.startswith('우') else 'L'
                pitch_summary['pitcher_team_code'] = pitcher_team_code
                pitch_summary['pitcher_name'] = pitcher_data.get('name', 'N/A')
            
                # 2. Batter Info (From Type 8 event)
                pitch_summary['batter_id'] = current_batter_id
                pitch_summary['batter_team_code'] = batter_team_code
                pitch_summary['batter_lineup_pos'] = current_batter_lineup_pos
                pitch_summary['batter_name'] = current_batter_name
    
                # 4. Pitch Details
                pitch_summary['is_throwing_stretch'] = (pitch_summary['base1'] != '0') or \
                                                     (pitch_summary['base2'] != '0') or \
                                                     (pitch_summary['base3'] != '0')
    
                
                pitch_summary['pitch_type'] = detail.get('stuff', 'N/A')
                pitch_summary['pitch_speed_kph'] = detail.get('speed', 'N/A')
                pitch_summary['pitch_result'] = detail.get('pitchResult', 'N/A')
                pitch_summary['is_outside_boundary'] = is_outside_boundary
                pitch_summary['zone_5x5_id'] = zone_5x5_id

                inn = pitch_summary['inn']
                pa_result_short = batter_record_lookup[current_batter_id].get(f'inn{inn}', 'N/A')

                pitch_summary['pa_result_long'] = pa_result_long
                pitch_summary['pa_result_short'] = pa_result_short
                pitch_summary['pa_result_base1'] = pa_result_base1
                pitch_summary['pa_result_base2'] = pa_result_base2
                pitch_summary['pa_result_base3'] = pa_result_base3
                
                processed_pitches_in_pa.append(pitch_summary)
            case TextType.PA_RESULT_SELF | TextType.PA_RESULT_RBI_SELF:
                break
            case TextType.INNING_START | TextType.SUBSTITUTION | TextType.TIMEOUT | \
                    TextType.PA_RESULT_RUNNER | TextType.PA_RESULT_RBI_RUNNER | \
                    TextType.INNING_END:
                pass
            case _:
                print(f"Unknown Text Type: {event_type} {detail.get('text')}")
    
    return processed_pitches_in_pa

def main_processing_script(game_id):
    batter_record_lookup, pitcher_record_lookup = get_record_json_file(game_id)
    master_pa_list, pitcher_lookup = get_json_files(game_id)
    away_code, home_code = extract_team_codes(game_id)

    if not master_pa_list:
        print("No plate appearances found. Exiting.")
        return pd.DataFrame()

    all_processed_pitches = []

    # 3. Process all Plate Appearances
    for i, pa_data in enumerate(master_pa_list):
        # print(f"Processing PA {i+1}/{len(master_pa_list)}...") # Optional: Debug print
        pitches_in_pa = process_plate_appearance(pa_data, pitcher_lookup, away_code, home_code, batter_record_lookup, pitcher_record_lookup)
        all_processed_pitches.extend(pitches_in_pa)

    print(f"Total pitches processed: {len(all_processed_pitches)}")

    # 4. Convert to Pandas DataFrame and Clean
    df = pd.DataFrame(all_processed_pitches)

    df.rename(columns={
        'pitchId': 'pitch_id',
        'crossPlateX': 'plate_x_ft',
        'ballcount': 'ball_count',
        'crossPlateY': 'plate_y_ft',
        'topSz': 'strikezone_top',
        'bottomSz': 'strikezone_btm',
        'stance': 'batter_stance',
        'inn': 'inning'
    }, inplace=True)
    
    # 5. Output the Results
    output_filename = f'./{CSV_FOLDER}/{game_id}_{CSV_SUFFIX}.csv'
    df.to_csv(output_filename, index=False, encoding='utf-8')
    
    print(f"\n--- Final Processed DataFrame Saved to {output_filename} ---")
    print("5 rows with key columns:")
    print(f"No. of Columns: {len(all_processed_pitches[0])}")
    for key, value in all_processed_pitches[0].items():
        print(f"{key}: {value}")
    
    return df
    
# --- Execution Block ---
if __name__ == '__main__':
    game_id = '20250929LTSK02025'
    #game_id = '20250930LTHH02025'
    
    # Execute the main function
    final_df = main_processing_script(game_id)

Max inning set to: 9

Finished data collection. Total PA lists collected: 90
Total pitches processed: 276

--- Final Processed DataFrame Saved to ./pitch_processed/20250929LTSK02025_processed.csv ---
5 rows with key columns:
No. of Columns: 46
pitchId: 250929_183021
inn: 1
ballcount: 1
crossPlateX: -0.259424
crossPlateY: 0.7083
topSz: 3.318
bottomSz: 1.609
vy0: -138.385
vz0: -5.71146
vx0: 6.3944
z0: 5.74282
y0: 50.0
x0: -1.49306
ax: -16.5538
ay: 34.083
az: -6.89725
stance: R
plate_z_ft: 3.0828
is_batter_home: False
home_score: 0
away_score: 0
strike: 1
ball: 0
out: 0
base1: 0
base2: 0
base3: 0
pitcher_id: 54833
pitcher_stance: R
pitcher_team_code: SK
pitcher_name: 앤더슨
batter_id: 52568
batter_team_code: LT
batter_lineup_pos: 1
batter_name: 한태양
is_throwing_stretch: False
pitch_type: 직구
pitch_speed_kph: 153
pitch_result: T
is_outside_boundary: False
zone_5x5_id: 42
pa_result_long: 볼넷
pa_result_short: 4구
pa_result_base1: 1
pa_result_base2: 0
pa_result_base3: 0


In [ ]:
import pandas as pd
import numpy as np

# --- Helper Functions for Validation ---

def calculate_pitcher_stats(df):
    """
    Calculates key pitching metrics (Pitch Count, K, BB, H) from pitch-level data.
    """
    # 1. Group by Pitcher and PA for final PA results
    pa_results = df.dropna(subset=['pa_result']).drop_duplicates(
        subset=['pitcher_id', 'batter_id', 'inn', 'batter_team_code'], 
        keep='last' # Keep the row containing the final PA result
    )

    # 2. Total Pitch Count (Simply count rows per pitcher)
    pitch_counts = df.groupby('pitcher_id')['pitchId'].count().reset_index(name='Pitch_Count')

    # 3. Plate Appearance Results (K, BB, H, AB)
    pitcher_pa_stats = pa_results.groupby('pitcher_id').agg(
        # HITS (안타) - Singles, Doubles, Triples, HR
        H=('pa_result_type', lambda x: (x.isin(['1B', '2B', '3B', 'HR'])).sum()),
        # WALKS (볼넷) - Base on Balls
        BB=('pa_result_type', lambda x: (x == 'BB').sum()),
        # STRIKEOUTS (삼진)
        K=('pa_result_type', lambda x: (x == 'K').sum()),
        # HIT BY PITCH (사구) - Assuming 'HBP' is the code for Hit By Pitch
        HBP=('pa_result_type', lambda x: (x == 'HBP').sum()),
        # TOTAL PLATE APPEARANCES (타석)
        PA=('pa_result_type', 'count')
    ).reset_index()
    
    # 4. Calculate Outs (K + GO + FO + SacFly/Bunt outs)
    # The 'O' is often recorded in pa_result_type for outs.
    # For a precise check, you'd need the final 'out' count, but PA - H - BB - HBP is a good proxy for outs+errors.
    pitcher_pa_stats['Outs_Recorded'] = pitcher_pa_stats['K'] + \
                                      pa_results[pa_results['pa_result_type'].isin(['GO', 'FO', 'SF', 'SAC'])].groupby('pitcher_id')['pa_result_type'].count().fillna(0).values


    # 5. Merge and Finalize
    pitcher_report = pitch_counts.merge(pitcher_pa_stats, on='pitcher_id', how='left')
    pitcher_report['player_name'] = pitcher_report['pitcher_id'].map(df.set_index('pitcher_id')['pitcher_name'].iloc[0])
    pitcher_report['team'] = pitcher_report['pitcher_id'].map(df.set_index('pitcher_id')['pitcher_team_code'].iloc[0])
    
    # Calculate At Bats (타수) for Batting Average validation: AB = PA - BB - HBP - SF - SAC
    pitcher_report['AB'] = pitcher_report['PA'] - pitcher_report['BB'] - pitcher_report['HBP'] - \
                           pitcher_pa_stats['pa_result_type'].apply(lambda x: (x.isin(['SF', 'SAC'])).sum())
    
    return pitcher_report[['team', 'pitcher_name', 'Pitch_Count', 'PA', 'AB', 'H', 'BB', 'K', 'HBP', 'Outs_Recorded']].sort_values(by='Pitch_Count', ascending=False)


def calculate_batter_stats(df):
    """
    Calculates key batter metrics (PA, AB, H, BB, K) from pitch-level data.
    """
    # Use the same PA result aggregation logic, but grouped by batter_id
    pa_results = df.dropna(subset=['pa_result_type']).drop_duplicates(
        subset=['pitcher_id', 'batter_id', 'inn', 'batter_team_code'], 
        keep='last'
    )
    
    batter_report = pa_results.groupby('batter_id').agg(
        # HITS (안타)
        H=('pa_result_type', lambda x: (x.isin(['1B', '2B', '3B', 'HR'])).sum()),
        # WALKS (볼넷)
        BB=('pa_result_type', lambda x: (x == 'BB').sum()),
        # STRIKEOUTS (삼진)
        K=('pa_result_type', lambda x: (x == 'K').sum()),
        # HIT BY PITCH (사구)
        HBP=('pa_result_type', lambda x: (x == 'HBP').sum()),
        # SACRIFICE FLY/BUNT (희생타)
        SF_SAC=('pa_result_type', lambda x: (x.isin(['SF', 'SAC'])).sum()),
        # TOTAL PLATE APPEARANCES (타석)
        PA=('pa_result_type', 'count')
    ).reset_index()

    # Calculate At Bats (타수): AB = PA - BB - HBP - SF - SAC
    batter_report['AB'] = batter_report['PA'] - batter_report['BB'] - batter